In [10]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [12]:
local_weights_file='/home/garimachahar/learnTensorflow/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model= InceptionV3(input_shape= (150,150,3), include_top= False, weights= None)

pre_trained_model.load_weights(local_weights_file)

#iterate through thae layers and lock every layer so that they are not trainable with this code
for layer in pre_trained_model.layers:
    layer.trainable= False

In [3]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [13]:
last_layer= pre_trained_model.get_layer('mixed10')
last_output= last_layer.output

In [14]:
from tensorflow.keras.optimizers import RMSprop
x= layers.Flatten()(last_output)
x= layers.Dense(1024, activation='relu')(x)
x= layers.Dense(1, activation= 'sigmoid')(x)

model= Model(pre_trained_model.input, x)
model.compile(optimizer= RMSprop(lr=0.0001), loss= 'binary_crossentropy', metrics= ['acc'])


In [15]:
import os
base_dir = '/home/garimachahar/learnTensorflow/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
#train_datagen updated to do image augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
#Training along with validation 
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,  # 2000 images = batch_size * steps
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 50 steps
Epoch 1/100
100/100 - 118s - loss: 0.7867 - acc: 0.6655 - val_loss: 0.2075 - val_acc: 0.9460
Epoch 2/100
100/100 - 98s - loss: 0.5240 - acc: 0.7455 - val_loss: 0.3123 - val_acc: 0.9290
Epoch 3/100
100/100 - 95s - loss: 0.4726 - acc: 0.7745 - val_loss: 0.4205 - val_acc: 0.9300
Epoch 4/100
100/100 - 95s - loss: 0.4673 - acc: 0.7790 - val_loss: 0.8770 - val_acc: 0.8820
Epoch 5/100
100/100 - 95s - loss: 0.4589 - acc: 0.7820 - val_loss: 0.3116 - val_acc: 0.9480
Epoch 6/100
100/100 - 95s - loss: 0.4745 - acc: 0.7885 - val_loss: 0.4141 - val_acc: 0.9300
Epoch 7/100
100/100 - 95s - loss: 0.4283 - acc: 0.8035 - val_loss: 0.3166 - val_acc: 0.9450
Epoch 8/100
100/100 - 97s - loss: 0.4128 - acc: 0.8150 - val_loss: 0.3510 - val_acc: 0.9460
Epoch 9/100
100/100 - 95s - loss: 0.4217 - acc: 0.8105 - val_loss: 0.4767 - val_acc: 0.9390
Epoch 10/100
100/100 - 96s - loss: 0.4143 - acc: 0.8085 - val_loss: 0.3571 - v

In [ ]:
import matplotlib.pyplot as plt
# Retrieve a list of accuracy results on training and validation data
# sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training and validation data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')

In [ ]:
Layers in NN can sometimes end up having similar weights and possibly
impact each other leading to overfitting. With a large model like this,
the risk of overfitting increases. so we dropout some layers.

In [ ]:
#if you train previous model(without dropout layer) it'll lead to overfitting(can be seen in graph of accuracy)
#REDEFINING MODEL WITH DROPOUT LAYER 
from tensorflow.keras.optimizers import RMSprop
x= layers.Flatten()(last_output)
x= layers.Dense(1024, activation='relu')(x)
#adding dropout to potentially avoid overfitting. Below line removes 20% layers from our model
#this has the effect of neighbours not affecting each other too much
x= layers.Dropout(0.2)(x)
x= layers.Dense(1, activation= 'sigmoid')(x)

model= Model(pre_trained_model.input, x)
model.compile(optimizer= RMSprop(lr=0.0001), loss= 'binary_crossentropy', metrics= ['acc'])

In [ ]:
#Training again with new model along with validation 
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,  # 2000 images = batch_size * steps
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2)

In [ ]:
# Retrieve a list of accuracy results on training and validation data
# sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training and validation data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')